<a href="https://colab.research.google.com/github/emmynuel/Data-Analysis--Partner/blob/main/GTb_Reconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive" | grep ABUJA

ABUJA DISCO COLLECTIONS SUSP (1).csv
ABUJA DISCO COLLECTIONS SUSP (2).csv
ABUJA DISCO COLLECTIONS SUSP (52).csv
ABUJA DISCO COLLECTIONS SUSP.csv


In [ ]:
import pandas as pd
# import mysql.connector
def pullDataFromBQ(query):
    project_id = 'buypower-mobile-app'
    df = pd.io.gbq.read_gbq(query, project_id=project_id)
    return df



In [ ]:
import glob
import os
import pandas as pd

folder_name = '/content/drive/My Drive'
file_type = 'csv'
print(glob.glob(folder_name + "/ABUJA*SUSP *(1)*."+file_type))

gt_data = pd.concat([pd.read_csv(f) for f in glob.glob(folder_name + "/ABUJA*SUSP *1)*."+file_type) ],ignore_index=True)

gt_data.shape

['/content/drive/My Drive/ABUJA DISCO COLLECTIONS SUSP (1).csv']


(122928, 7)

In [ ]:
gt_data = pd.read_csv('/content/drive/My Drive/ABUJA DISCO COLLECTIONS SUSP (2).csv')
gt_data.head()

,Trans Date,Reference,Value Date,Debit,Credit,Balance,Remarks
0,04-Jan-2022,0.0,02-Jan-2022,,1778.40,1778.4,OWN ACCOUNT TRANSFER 335642735166|25113877598|...
1,04-Jan-2022,0.0,02-Jan-2022,,4950.00,6728.4,OWN ACCOUNT TRANSFER 424438281211|04176783910|...
2,04-Jan-2022,0.0,03-Jan-2022,,2970.00,9698.4,OWN ACCOUNT TRANSFER 511677924767|010119000521...
3,04-Jan-2022,0.0,03-Jan-2022,,1980.00,11678.4,OWN ACCOUNT TRANSFER 827315817764|04136359439|...
4,04-Jan-2022,0.0,03-Jan-2022,,1086.80,12765.2,OWN ACCOUNT TRANSFER 413299646617|04181413461|...


In [ ]:
bp_gt_data = pd.read_csv('/content/drive/My Drive/buypower__2021-12-17_to_2022-01-03.csv')
bp_gt_data

,Date,Ref,Phone,Amount,DisCo,token,Meter,Remarks_x
0,2021-12-17 00:05:59,234987453883,_08036360067,10000.0,IBADAN,_28629509422683824709,_62141359232,234987453883 | 62141359232 | 08036360067 | | ...
1,2021-12-17 00:07:05,484217353379,_08057536053,1000.0,IBADAN,_48184052194348869018,_62418682118,484217353379 | 62418682118 | 08057536053 | | ...
2,2021-12-17 00:10:11,886162572734,_08162323782,2100.0,ABUJA,_73250009087039793877,_04182698813,886162572734 | 04182698813 | 08162323782 | Und...
3,2021-12-17 00:12:03,578127449416,_08121344863,400.0,IBADAN,_39939748123771383373,_54140821858,578127449416 | 54140821858 | 08121344863 | OSU...
4,2021-12-17 00:13:47,217346619627,_08035878614,5100.0,ABUJA,_44189362553058783103,_14200000488,217346619627 | 14200000488 | 08035878614 | NO....
...,...,...,...,...,...,...,...,...
15729,2022-01-03 23:57:03,946346364953,_08134319669,2100.0,ABUJA,_01417299530340450161,_07083084298,946346364953 | 07083084298 | 08134319669 | | ...
15730,2022-01-03 23:57:03,256264181942,_07034593959,900.0,IBADAN,_29268736420118590488,_0159001369952,256264181942 | 0159001369952 | 07034593959 | ...
15731,2022-01-03 23:58:58,183355651559,_08166901777,2000.0,IBADAN,_71988362725673101562,_62141258749,"183355651559 | 62141258749 | 08166901777 | 3, ..."
15732,2022-01-03 23:59:06,644165784551,_08167551848,5000.0,IBADAN,_58771168147734348698,_62180123572,644165784551 | 62180123572 | 08167551848 | | ...


In [ ]:
gt_data.head()

,Trans Date,Reference,Value Date,Debit,Credit,Balance,Remarks
0,23-Nov-2021,0.0,21-Nov-2021,,1980.00,1980.0,OWN ACCOUNT TRANSFER 889815559698|07081116928|...
1,23-Nov-2021,0.0,21-Nov-2021,,4950.00,6930.0,OWN ACCOUNT TRANSFER 827234749939|013720010922...
2,23-Nov-2021,0.0,21-Nov-2021,,990.00,7920.0,OWN ACCOUNT TRANSFER 932425563929|45700442358|...
3,23-Nov-2021,0.0,21-Nov-2021,,1970.00,9890.0,OWN ACCOUNT TRANSFER 881688627271|30530291431|...
4,23-Nov-2021,0.0,21-Nov-2021,,990.00,10880.0,OWN ACCOUNT TRANSFER 884586866165|62120100656|...


In [ ]:

i = []
j = []

for x in range(gt_data['Trans Date'].size):
    if len(str(gt_data.loc[x,'Trans Date'])) >= 20:
        i.append(gt_data.loc[x,'Trans Date'])
        j.append(x)
        

f = pd.DataFrame(i)
f.to_csv('Error Remarks.csv',index=False)
gt_data = gt_data.drop(j).reset_index(drop=True)
gt_data.head()

,Trans Date,Reference,Value Date,Debit,Credit,Balance,Remarks
0,15-Dec-2021,0.0,14-Dec-2021,,19760.00,19760.0,OWN ACCOUNT TRANSFER 651214875771|45022111988|...
1,15-Dec-2021,0.0,14-Dec-2021,,1485.00,21245.0,737 MERCHANT PAYMENTS 492198262972|45701387982...
2,15-Dec-2021,0.0,14-Dec-2021,,9900.00,31145.0,737 MERCHANT PAYMENTS 538878799728|01790001671...
3,15-Dec-2021,0.0,14-Dec-2021,,4940.00,36085.0,OWN ACCOUNT TRANSFER 433869781715|010116039332...
4,15-Dec-2021,0.0,14-Dec-2021,,9850.00,45935.0,OWN ACCOUNT TRANSFER 231769814298|04185339027|...


In [ ]:
#Preserve this format, thats the format GT Bank wants

start_date = '2021-12-17'
end_date = '2022-01-03'
query = f'''







with all_payments as 

    (SELECT timestamp_trunc(datetime(pt.created_at, "Africa/Lagos") ,second) as Date, vr.payment_reference as Ref, pt.phone as Phone, 
    cast(pt.amount as float64) as Amount,
    case when pt.disco_code in ("PHISBOND", "PROTOGY", "ACCESSPOWER", "BH") then "ABUJA" else pt.disco_code end as  DisCo,  vr.token as token, pt.meter_no as Meter ,pt.payment_type , pt.channel , 
    pt.generated_by , pt.order_id , pt.meter_no 
    FROM views.payment_transactions pt join views.transaction_vend_response vr on pt.order_id = payment_reference 
    where date(pt.created_at, "Africa/Lagos") between "{start_date}" and "{end_date}" ),


online_offline as 
    (select * from all_payments where payment_type in ('ONLINE','OFFLINE')), 
    
apix_ as 
    (select * from all_payments where channel = "APIX" and generated_by = 1253797 and payment_type not in ('ONLINE','OFFLINE')),
    
cleaned_data as (
select pt.* except (rn), concat(pt.order_id  ,' | ', pt.meter_no   , ' | ', pt.phone ,' | ',
case when ms.customer_address is null then '' else ms.customer_address end  , ' | ', pt.amount , ' | ', pt.payment_type ) as Remarks  
from 
  (
    select *, ROW_NUMBER() over(partition by order_id )  as rn 
    from 
      (
        select * from apix_ ap 
        union all 
        (select * from online_offline) 
       ) 
   ) pt 
  left join 
  (
   select * 
   from 
      (
        select  meter_no, customer_address,  ROW_NUMBER() over(partition by meter_no )  as rnn 
        from dev_data.meter_store
      ) 
    where rnn = 1
   )
 ms on ms.meter_no = pt.meter_no 
 and rn =1
),

corrected_data as (

  select * except (rn, payment_type, channel, generated_by, order_id, meter_no) from (
    select *, ROW_NUMBER() over (partition by Ref) as rn 
    from cleaned_data 
    )
  where rn = 1
  )

select *  from corrected_data
order by Date asc






'''.format(start=start_date,end=end_date)

bp_gt_data = pullDataFromBQ(query)
bp_gt_data.head()

NameError: ignored

In [ ]:
bp_gt_data.Date.max()

Timestamp('2022-01-03 23:59:13')

In [ ]:
#Pull out payment reference, amount and remarks from gt's data. payment reference and order id are used 
#interchangably in buypower

def helper(arr):
    x = str(arr).split('|')
    return x[0].split()[-1]
# gt_data_temp = gt_data[['CREDIT_AMT','REMARKS','TRA_DATE']]

gt_data_temp = gt_data[['Credit','Remarks','Trans Date']]

gt_data_temp['ORDER_ID'] = gt_data_temp['Remarks'].apply(helper)
# gt_data_temp.columns = ['Credit', 'REMARKS', 'Trans Date', 'ORDER_ID']
gt_data_temp.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Credit,Remarks,Trans Date,ORDER_ID
13629,9880.00,OWN ACCOUNT TRANSFER 745316179582|54170017351|...,05-Jan-2022,745316179582
13630,1485.00,OWN ACCOUNT TRANSFER 143897236543|47300053841|...,05-Jan-2022,143897236543
13631,2462.50,OWN ACCOUNT TRANSFER 938636734772|58000059830|...,05-Jan-2022,938636734772
13632,1477.50,OWN ACCOUNT TRANSFER 794486254376|012820116166...,05-Jan-2022,794486254376
13633,,STANDING ORDER DAILY SI SWEEP FRM GTB IMPERSON...,05-Jan-2022,TEC


In [ ]:
bp_gt_data.Ref = bp_gt_data.Ref.astype(str)

In [ ]:
#Merge the two dfs
all_data = pd.merge(bp_gt_data,gt_data_temp,how='outer',left_on='Ref',right_on='ORDER_ID',indicator=True)

In [ ]:
#Seperate into three DFs
both_have = all_data[all_data['_merge'] == 'both']
bp_only = all_data[all_data['_merge'] == 'left_only']
gt_only = all_data[all_data['_merge'] == 'right_only']

In [ ]:
def get_date(x):
    value = 0
    try:
        value = int(x.split('-')[0])
    except: pass
    return value

In [ ]:
gt_only[(gt_only['Trans Date'].apply(get_date)  <= 16) & (~gt_only['Trans Date'].str.contains('Jan'))]

,Date,Ref,Phone,Amount,DisCo,token,Meter,Remarks_x,Credit,Remarks_y,Trans Date,ORDER_ID,_merge
38693,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19760.00,OWN ACCOUNT TRANSFER 651214875771|45022111988|...,15-Dec-2021,651214875771,right_only
38694,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1485.00,737 MERCHANT PAYMENTS 492198262972|45701387982...,15-Dec-2021,492198262972,right_only
38695,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9900.00,737 MERCHANT PAYMENTS 538878799728|01790001671...,15-Dec-2021,538878799728,right_only
38696,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4940.00,OWN ACCOUNT TRANSFER 433869781715|010116039332...,15-Dec-2021,433869781715,right_only
38697,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9850.00,OWN ACCOUNT TRANSFER 231769814298|04185339027|...,15-Dec-2021,231769814298,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2970.00,737 MERCHANT PAYMENTS 878287881616|01372000294...,16-Dec-2021,878287881616,right_only
43796,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1976.00,OWN ACCOUNT TRANSFER 563543451756|012400104987...,16-Dec-2021,563543451756,right_only
43797,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980.00,OWN ACCOUNT TRANSFER 559224875491|58000617363|...,16-Dec-2021,559224875491,right_only
48116,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.|No1chief Fred,nan,right_only


In [ ]:
print(bp_only.shape, gt_only.shape, both_have.shape)

(3071, 13) (971, 13) (12663, 13)


In [ ]:
bp_only[bp_only.DisCo.isin(["ACCESSPOWER","BH", "Protogy", "PhisBond"])]

,Date,Ref,Phone,Amount,DisCo,token,Meter,Remarks_x,Credit,Remarks,Trans Date,ORDER_ID,_merge


In [ ]:
bp_only

,Date,Ref,Phone,Amount,DisCo,token,Meter,Remarks_x,Credit,Remarks,Trans Date,ORDER_ID,_merge
0,2021-12-17 00:05:59,234987453883,_08036360067,10000.0,IBADAN,_28629509422683824709,_62141359232,234987453883 | 62141359232 | 08036360067 | | ...,NaN,NaN,NaN,NaN,left_only
1,2021-12-17 00:07:05,484217353379,_08057536053,1000.0,IBADAN,_48184052194348869018,_62418682118,484217353379 | 62418682118 | 08057536053 | | ...,NaN,NaN,NaN,NaN,left_only
3,2021-12-17 00:12:03,578127449416,_08121344863,400.0,IBADAN,_39939748123771383373,_54140821858,578127449416 | 54140821858 | 08121344863 | OSU...,NaN,NaN,NaN,NaN,left_only
5,2021-12-17 00:16:37,553383351554,_08067724141,3900.0,IBADAN,_52443485014069466204,_01455223097,553383351554 | 01455223097 | 08067724141 | | ...,NaN,NaN,NaN,NaN,left_only
6,2021-12-17 00:23:26,667362913919,_07066312261,900.0,IBADAN,_24655909727110282291,_14201840601,667362913919 | 14201840601 | 07066312261 | | ...,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15720,2022-01-03 23:32:37,242218943267,_08033755354,2000.0,IBADAN,_40503139884258115745,_62141105858,242218943267 | 62141105858 | 08033755354 | | ...,NaN,NaN,NaN,NaN,left_only
15730,2022-01-03 23:57:03,256264181942,_07034593959,900.0,IBADAN,_29268736420118590488,_0159001369952,256264181942 | 0159001369952 | 07034593959 | ...,NaN,NaN,NaN,NaN,left_only
15731,2022-01-03 23:58:58,183355651559,_08166901777,2000.0,IBADAN,_71988362725673101562,_62141258749,"183355651559 | 62141258749 | 08166901777 | 3, ...",NaN,NaN,NaN,NaN,left_only
15732,2022-01-03 23:59:06,644165784551,_08167551848,5000.0,IBADAN,_58771168147734348698,_62180123572,644165784551 | 62180123572 | 08167551848 | | ...,NaN,NaN,NaN,NaN,left_only


In [ ]:
#Select required columns and prepend underscore to prevent excel deleting leading zeros from columns
cols = ['Date', 'Ref', 'Phone', 'Amount', 'DisCo', 'token', 'Meter', 'Remarks_x']
bp_only = bp_only[cols]
bp_only['Phone'] = '_' + bp_only['Phone']
bp_only['token'] = '_' + bp_only['token']
bp_only['Meter'] = '_' + bp_only['Meter']
bp_only = bp_only.sort_values(by='Date')
bp_only.to_csv('buypower__{}_to_{}new.csv'.format(start_date,end_date),index=False)


In [ ]:
bp_only['Amount'].sum()

11403534.0

In [ ]:
!cp /content/buypower__2021-12-17_to_2022-01-03new.csv /content/drive/My\ Drive/

In [ ]:
#select required columns and remove noise rows from GT data eg "DAILY SI SWEEP FRM GTB IMPERSONAL LEDGERS TO  DATAPEX DRIFTY TEC"

gt_only = gt_only[['ORDER_ID','TRA_DATE','CREDIT_AMT','REMARKS']]
refs = [x for x in gt_only['ORDER_ID'].tolist() if len(x) < 20]
#Extract references and check bigquery for those transactions details if they exist and the date when they occurred
query = '''
select date(timestamp_millis(vr.created_at)) as buypower_date, payment_reference from `dev_data.transaction_vend_response` vr where vr.payment_reference in {}
'''.format(tuple(refs))
temp = pullDataFromBQ(query)

#Format columns properly and write to disk
gt_only = pd.merge(gt_only,temp,left_on='ORDER_ID',right_on='payment_reference')
gt_only = gt_only[['ORDER_ID',	'TRA_DATE','buypower_date',	'CREDIT_AMT',	'REMARKS']]
gt_only.to_csv('gt_only_july_1_to_july_31.csv',index=False)
gt_only.head()

[]


,ORDER_ID,TRA_DATE,buypower_date,CREDIT_AMT,REMARKS
0,621318333449,1/7/20 0:00,2020-07-01,494.0,621318333449|45051187347|ENUGU|PREPAID|200|Ike...
1,415989295448,1/7/20 0:00,2020-07-01,988.0,415989295448|0124000613255|PH|PREPAID|200|UMO ...
2,959366836967,1/7/20 0:00,2020-07-01,1976.0,959366836967|01455346468|PH|PREPAID|200|HON SU...
3,141846254846,1/7/20 0:00,2020-07-01,2964.0,141846254846|45028883820|ENUGU|PREPAID|200|Oko...
4,797226257329,1/7/20 0:00,2020-07-01,691.6,797226257329|610124001133931|PH|PREPAID|200|MR...


In [ ]:
#Download the csv files and send to tosin and paschal or copy to drive

In [ ]:
!cp '/content/buypower__2021-09-4_to_2021-10-19.csv' '/content/drive/MyDrive'

In [ ]:
"Knearestneighbor".split('n')

['K', 'earest', 'eighbor']

In [ ]:
bp_only[bp_only['token']=='000']

,Date,Ref,Phone,Amount,DisCo,token,Meter,Remarks_x


In [ ]:
bq_data = pullDataFromBQ('select payment_reference, count(distinct token )  num_of_token from dev_data.transaction_vend_response where date(timestamp_millis(created_at)) between "2021-05-1" and "2021-05-27" group by payment_reference')

In [ ]:
gt_data = pd.read_excel("/content/GTBANK VARIANCE.xlsx", skiprows=1)

In [ ]:
gt_data['Order Id'] = gt_data['Order Id'].astype(str)

In [ ]:
check = pd.merge(gt_data, bq_data, left_on = "Order Id", right_on = "payment_reference", how= "left", indicator = True)

In [ ]:
check[check._merge == 'both'][check.num_of_token >1 ]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Date,Order Id,Transaction Counts,Total Amount Paid,Exposure,payment_reference,num_of_token,_merge


In [ ]:
bp_only

,Date,Ref,Phone,Amount,DisCo,token,Meter,Remarks_x
677,2021-10-03 13:35:59,285258927226,_08025350518,6000.0,PH,_30083387743150084940,_0124000736767,285258927226 | 0124000736767 | 08025350518 | ...
895,2021-10-03 15:40:40,218674215829,_08036320277,500.0,KADUNA,_22952179121583029091,_95300221637,218674215829 | 95300221637 | 08036320277 | FLT...
897,2021-10-03 15:43:22,824466617455,_08139511796,1000.0,IBADAN,_66310220603285683767,_45050649263,824466617455 | 45050649263 | 08139511796 | | ...
1006,2021-10-03 16:46:43,446738447192,_08035335314,3000.0,KADUNA,_63592061047200037063,_95500036520,446738447192 | 95500036520 | 08035335314 | 5 P...
1129,2021-10-03 17:50:10,747221436963,_08032900661,500.0,PH,_12346131344787485496,_0124000670024,747221436963 | 0124000670024 | 08032900661 | I...
...,...,...,...,...,...,...,...,...
51901,2021-10-26 23:41:08,523596428392,_07033774021,2100.0,ABUJA,_59674310246366122592,_45060197584,523596428392 | 45060197584 | 07033774021 | | ...
51903,2021-10-26 23:42:31,925827942574,_08023406761,1000.0,IBADAN,_27501106531900188306,_14260467262,925827942574 | 14260467262 | 08023406761 | | ...
51907,2021-10-26 23:51:18,656747696236,_07051441280,7100.0,ABUJA,_07297295699001625675,_07127021157,656747696236 | 07127021157 | 07051441280 | | ...
51910,2021-10-26 23:56:36,219636825455,_08121733305,1600.0,ABUJA,_000,_45701454592,219636825455 | 45701454592 | 08121733305 | | ...


In [ ]:
bp_gt_data

,Date,Ref,Phone,Amount,DisCo,token,Meter,Remarks
0,2021-09-04 00:01:30,968311225186,09099119990,5100.0,ABUJA,66380704568593027758,95500043195,968311225186 | 95500043195 | 09099119990 | | ...
1,2021-09-04 00:02:46,178276343321,08036391630,10100.0,ABUJA,29654901323445625626,04160110831,178276343321 | 04160110831 | 08036391630 | | ...
2,2021-09-04 00:05:56,995719945734,08068831991,4000.0,PH,39364645854528040739,04177240803,995719945734 | 04177240803 | 08068831991 | RD ...
3,2021-09-04 00:08:10,618542581765,07030134593,500.0,JOS,19444286555080198981,54161026320,618542581765 | 54161026320 | 07030134593 | | ...
4,2021-09-04 00:09:22,179446595469,08032106131,20000.0,KANO,26312791913928392414,30510098806,179446595469 | 30510098806 | 08032106131 | | ...
...,...,...,...,...,...,...,...,...
86086,2021-10-19 23:56:07,966678569958,08030630795,1600.0,JOS,57198413692611301986,0101160358160,966678569958 | 0101160358160 | 08030630795 | ...
86087,2021-10-19 23:57:31,874782241581,07033361795,2900.0,IBADAN,67096221520627365961,62418508453,874782241581 | 62418508453 | 07033361795 | NO ...
86088,2021-10-19 23:58:03,617486286465,08064452394,3000.0,ENUGU,60981344219509760643,45060447013,617486286465 | 45060447013 | 08064452394 | 22 ...
86089,2021-10-19 23:58:06,428589337185,08023166971,5100.0,ABUJA,07117496856404539220,04178551893,428589337185 | 04178551893 | 08023166971 | | ...
